# Lexical-Sample Supervised Word Sense Disambiguation

## Progress
Classifier: Linear SVM
Cross validation: k-fold cross validation

### Include PERSON, LOCATION, ORGANIZATION, OTHER ENTITY, and MWE
5-fold CV

Features, cross validation macro average accuracy:
- It Makes Sense's Local Collocation SVD + Surrounding Words SVD: .74
- It Makes Sense's Local Collocation + Surrounding Words: .726
- It Makes Sense's Local Collocation SVD: .721
- Latent Semantic Analysis: .713
- It Makes Sense's Local Collocation: .71
- Surrounding Words SVD: .708
- Collocation Vector: .70
- TF-IDF: .70
- Unigram-Bigram TFIDF: .69
- Choose most frequent sense: .57


Features, cross validation macro average F1-score:
- It Makes Sense's Local Collocation SVD + Surrounding Words SVD: .533
- It Makes Sense's Local Collocation SVD: .528
- It Makes Sense's Local Collocation + Surrounding Words: .526
- It Makes Sense's Local Collocation: .524
- Collocation Vector SVD: .506
- Collocation Vector: .495
- Latent Semantic Analysis: .491
- Surrounding Words SVD: .477
- TF-IDF: .427
- Unigram-Bigram TF-IDF: .399
- Choose most frequent sense: .220

### Pure WSD,  with MWE
5-fold CV
- It Makes Sense's Local Collocation SVD + Surrounding Words SVD: .550
- It Makes Sense's Local Collocation SVD: .539
- It Makes Sense's Local Collocation: .537
- Latent Semantic Analysis: .506
- Surrounding Words SVD: .505
- Collocation Vector SVD: .505
- Collocation Vector: .501
- TF-IDF: .434
- Choose most frequent sense: .241

### Pure WSD,  not even MWE
5-fold CV, F1-score
- Iacobacci, et. al (2016) replication: .635
- Wikipedia word embedding + POS tags SVD + IMS collocation vectors: .635
- Wikipedia word embedding + POS tags SVD: .630
- Wikipedia word embedding + POS tags SVD + Surrounding words SVD: .630
- Wikipedia word embedding + IMS collocation vectors + Surrounding words SVD: .630
- Wikipedia word embedding + IMS collocation vectors: .628
- Wikipedia word embedding + Surrounding words SVD: .623
- Wikipedia word embedding: .618
- It Makes Sense's (Zhong & Ng, 2010) Replication, but SVD: .576
- It Makes Sense's Local Collocation SVD + Surrounding Words SVD: .567
- POS Tags SVD + Surrounding Words SVD: .561
- It Makes Sense's Local Collocation SVD + POS Tags SVD: .558
- It Makes Sense's Local Collocation SVD: .546
- It Makes Sense's Local Collocation: .545
- Latent Semantic Analysis: .524
- Surrounding Words SVD: .520
- POS Tags SVD: .488
- POS Tags: .488
- TF-IDF: .459
- Choose most frequent sense: .253

In general, training accuracy / f1-score is perfect, but the cross validation score is way too low, which means:
**Overfit**

- Now with word embedding, the training score is never 100% perfect and the cross validation score *improved drastically*

### TODO
- Tackle overfit problem -> word embedding somewhat solved this
- Wikipedia Indonesia Word Embedding -> done
- SVD with larger dimension (with extra memory)
- Build balanced dataset: manual labor

In [14]:
import sys
import pandas as pd
import numpy as np

# Load Data

In [2]:
dataset = pd.read_csv('train_data.csv')
dataset.head()

,Unnamed: 0,kata,sense,kalimat,pos_tags,clean,targetpos_clean,targetpos_ori,targetpos_pos_tag
0,0,cerah,4801,cuaca cerah adalah lazim panjang tahun,NN NN VB NN NN NN Z,cuaca cerah lazim,1,1,1
1,1,cerah,4801,gambar yang hasil oleh layarnya cukup cerah da...,NNP SC VB IN NN RB JJ CC VB NN SC JJ VB NN SC ...,gambar hasil layarnya cerah milik speaker hasi...,3,6,6
2,2,cerah,4803,masa depan yang cerah bagi pemuda umur somenum...,NN NN SC VB IN NN NN CD IN NNP NNP CD Z,cerah bagi pemuda umur prancis abad,0,3,3
3,3,cerah,4801,cor caroli alpha canum venaticorum nama lengka...,NNP NNP Z NNP NNP NNP Z Z Z NN RB VB NNP NNP N...,cor caroli alpha canum venaticorum nama lengka...,12,16,21
4,4,cerah,4801,sanders lebih suka cat air untuk lilo dengan m...,NN RB VB NN NN SC NNP IN NN VB NN NN NN NN NN Z,sanders suka cat air lilo maksud tampil warna ...,8,11,11


# Drop rare sense from training set

In [7]:
RARE_LIMIT = 5
sense_set = set(dataset.sense)

In [8]:
rare_sense = set(filter(lambda s: len(dataset.query('sense == "{}"'.format(s))) <= RARE_LIMIT, sense_set))
len(rare_sense)

37

In [9]:
dataset_kata = []
dataset_sense = []
dataset_kalimat = []
dataset_clean = []
dataset_pos_clean = []
dataset_pos_ori = []
dataset_pos_tags = []
dataset_pos_pos_tag = []
for i in range(len(dataset)):
    row = dataset.iloc[i]
    if row.sense not in rare_sense:
        dataset_kata.append(row.kata)
        dataset_sense.append(row.sense)
        dataset_kalimat.append(row.kalimat)
        dataset_clean.append(row.clean)
        dataset_pos_clean.append(row.targetpos_clean)
        dataset_pos_ori.append(row.targetpos_ori)
        dataset_pos_tags.append(row.pos_tags)
        dataset_pos_pos_tag.append(row.targetpos_pos_tag)

dataset = pd.DataFrame({
    'kata': dataset_kata,
    'sense': dataset_sense,
    'kalimat': dataset_kalimat,
    'clean': dataset_clean,
    'targetpos_clean': dataset_pos_clean,
    'targetpos_ori': dataset_pos_ori,
    'pos_tags': dataset_pos_tags,
    'targetpos_pos_tag': dataset_pos_pos_tag,
})

In [10]:
set(dataset.query('kata == "{}"'.format('panas')).sense)

{'4901', '4903', '4904'}

In [11]:
len(dataset)

8311

# Feature Extraction

## POS Tags

In [86]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from functools import reduce

In [87]:
POS_TAGS_WINDOW = 2

In [88]:
pos_tags = [['-' for j in range(2*POS_TAGS_WINDOW+1)] for i in range(len(dataset))]
possible_tags = set()

for i in range(len(dataset)):
    row = dataset.iloc[i]
    tags = row.pos_tags.split()
    position = row.targetpos_pos_tag
    pos_tags[i][POS_TAGS_WINDOW] = tags[position]
    j = position-1
    k = POS_TAGS_WINDOW - 1
    while j >= 0 and j >= position - POS_TAGS_WINDOW:
        if tags[j] == 'Z':
            break # do not even include
        pos_tags[i][k] = tags[j]
        k -= 1
        j -= 1
    j = position+1
    k = POS_TAGS_WINDOW + 1
    while j < len(tags) and j <= position + POS_TAGS_WINDOW:
        pos_tags[i][k] = tags[j]
        if tags[j] == 'Z':
            break # include, then break

        k += 1
        j += 1
    
    

In [89]:
pos_tag_transformer = OneHotEncoder().fit(pos_tags)

In [90]:
pos_tags = pos_tag_transformer.transform(pos_tags)

In [91]:
pos_tags

<8311x98 sparse matrix of type '<class 'numpy.float64'>'
	with 41555 stored elements in Compressed Sparse Row format>

## TF-IDF

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [131]:
tfidf_u = TfidfVectorizer()
u_tfidf = tfidf_u.fit_transform(dataset.clean)

In [132]:
u_tfidf

<8311x20337 sparse matrix of type '<class 'numpy.float64'>'
	with 99962 stored elements in Compressed Sparse Row format>

## Unigram-Bigram TF-IDF
as in Faisal, et. al (2018) "Word Sense Disambiguation in Bahasa Indonesia using SVM"

In [109]:
combined_unigram_bigram = []

for i in range(len(dataset)):
    row = dataset.iloc[i]
    combined_unigram_bigram.append(row.clean + ' ' + row.clean_bigram)

In [111]:
tfidf_ub = TfidfVectorizer()
ub_tfidf = tfidf_ub.fit_transform(combined_unigram_bigram)

In [112]:
ub_tfidf

<8721x109586 sparse matrix of type '<class 'numpy.float64'>'
	with 220345 stored elements in Compressed Sparse Row format>

## Latent Semantic Analysis

In [94]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

In [136]:
svdtfidf = make_pipeline(TruncatedSVD(1000), Normalizer(copy=False))
lsa = svdtfidf.fit_transform(u_tfidf)

In [137]:
lsa.shape

(8311, 1000)

## Collocation Vector

In [101]:
from sklearn.feature_extraction.text import CountVectorizer
from preprocessor import normalize_money, normalize_number, stemmer, pipe

In [41]:
CONTEXT_WINDOW = 3

In [42]:
context_words = [[] for i in range(len(dataset))]

for i in range(len(dataset)):
    tokens = dataset.iloc[i].kalimat.split()
    pos = dataset.iloc[i].targetpos_ori
    for j in range(max(0, pos-CONTEXT_WINDOW), pos):
        token = pipe(normalize_money, normalize_number, stemmer.stem)(tokens[j])
        context_words[i].append(token)
    for j in range(pos+1, min(len(tokens), pos+CONTEXT_WINDOW+1)):
        token = pipe(normalize_money, normalize_number, stemmer.stem)(tokens[j])
        context_words[i].append(token)
    context_words[i] = ' '.join(context_words[i])

In [43]:
cv = CountVectorizer()
collocation_vector = cv.fit_transform(list(map(lambda s: ' '.join(set(s.split())), context_words)))

In [44]:
collocation_vector

<8428x7855 sparse matrix of type '<class 'numpy.int64'>'
	with 45075 stored elements in Compressed Sparse Row format>

## Correct implementation of collocation vectors
as in Zhong & Ng (2010) "It Makes Sense", but unigram and bigrams only 

In [109]:
from scipy.sparse.csr import csr_matrix
import time
from functools import reduce

In [110]:
collocation_pos = {
    (-2, -2), (-1, -1), (1, 1), (2, 2), (-2, -1), (-1, 1), (1, 2),
}

In [111]:
def get_collocation(sentence, targetpos, L, R):
    col = ['-' for i in range(R-L+1 - (1 if L < 0 and R > 0 else 0))]
    tokens = sentence.split()
    L = targetpos+L
    R = targetpos+R
    j = L
    i = 0
    while j <= R:
        if j < 0:
            j += 1
            i += 1
            continue
        if j == targetpos:
            j += 1
            continue
        if j >= len(tokens):
            break
        col[i] = tokens[j]
        j += 1
        i += 1
    
    return col

In [112]:
print(dataset.iloc[2].kalimat, dataset.iloc[2].targetpos_ori)

masa depan yang cerah bagi pemuda umur somenumber di prancis abad somenumber 3


In [113]:
collocation_words = [[] for i in range(len(dataset))]
collocations = ['' for i in range(len(dataset))]

for i in range(len(dataset)):
    instance = dataset.iloc[i]
    for l, r in collocation_pos:
        collocation_words[i].append(get_collocation(instance.kalimat, instance.targetpos_ori, l, r))
        collocations[i] = ' '.join(list(map(lambda s: ' '.join(s) , collocation_words[i])))

In [114]:
cv_unigram_bigram = CountVectorizer().fit(collocations)

In [115]:
collocation_vectors = [0 for i in range(len(dataset))]

for i in range(len(dataset)):
    collocation_vectors[i] = cv_unigram_bigram.transform(
        reduce(lambda acc, nex: [*acc, *nex], collocation_words[i], [])
    ).reshape(1, -1)

In [116]:
collocation_vectors = csr_matrix([np.array(vec.toarray()[0], dtype=np.bool) for vec in collocation_vectors])

In [117]:
collocation_vectors

<8311x62350 sparse matrix of type '<class 'numpy.bool_'>'
	with 80193 stored elements in Compressed Sparse Row format>

## Surrounding Words

In [102]:
cv_bin = CountVectorizer()
surrounding_words = cv_bin.fit_transform(
    list(map(lambda s: ' '.join(set(s.split())), dataset.clean))
)

In [103]:
surrounding_words = csr_matrix(np.array([surrounding_words[i].toarray()[0] for i in range(surrounding_words.shape[0])], dtype=np.bool))

In [104]:
surrounding_words

<8311x20337 sparse matrix of type '<class 'numpy.bool_'>'
	with 99962 stored elements in Compressed Sparse Row format>

## Wikipedia Word2Vec Word Embedding

In [12]:
import gensim

In [80]:
EMBEDDING_SIZE = 50

In [81]:
word_vectors = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format('../wikipedia_indonesia_embedding{}.model'.format(EMBEDDING_SIZE))

### Exponential Decay Word Embedding Features
Iacobacci, et. al (2016)

In [82]:
embedding = []

W = 5
alpha = 1 - (np.power(0.1, np.power(W-1.0, -1)))

for p in range(len(dataset)):
    if (p % 800) == 0:
        sys.stdout.write("\r{0:.2f} %".format(p/len(dataset)))
        sys.stdout.flush()
    instance = dataset.iloc[p]
    e = np.zeros(EMBEDDING_SIZE)
    I = instance.targetpos_ori
    words = instance.kalimat.split()
    for i in range(EMBEDDING_SIZE):
        for j in range(max(0, I-W), min(len(words), I+W+1)):
            if j == I:
                continue
            try:
                e[i] += (word_vectors.get_vector(words[j])[i] * (np.power(1 - alpha, abs(I-j) - 1)))
            except:
                continue
    embedding.append(e)
            

0.96 %

In [83]:
embedding = np.array(embedding)

In [84]:
embedding.shape

(8311, 50)

# Form Training Set

## It Makes Sense's Collocation Vectors only

In [127]:
X_train = collocation_vectors

In [128]:
collocation_vectors

<8311x62350 sparse matrix of type '<class 'numpy.bool_'>'
	with 80193 stored elements in Compressed Sparse Row format>

## IMS Collocation Vectors SVD

In [118]:
svdimscv = make_pipeline(TruncatedSVD(5000), Normalizer(copy=False))

In [119]:
begin = time.perf_counter()
X_train = svdimscv.fit_transform(collocation_vectors)
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 829.8634827000005


In [120]:
imscvsvd = X_train

In [121]:
X_train = imscvsvd

In [122]:
X_train.shape

(8311, 5000)

## Surrounding Words Only

In [84]:
X_train = surrounding_words

## Surrounding Words SVD

In [105]:
svdsw = make_pipeline(TruncatedSVD(1000), Normalizer(copy=False))

In [106]:
begin = time.perf_counter()
swsvd = svdsw.fit_transform(surrounding_words)
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 27.190810500000225


In [42]:
X_train = swsvd

## IMS Collocation Vectors SVD + Surrounding Words SVD

In [43]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*imscvsvd[i], *swsvd[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 9.209585700000162


In [44]:
X_train.shape

(8311, 6000)

## Collocation Vector only

In [13]:
X_train = collocation_vector

## Collocation Vector SVD

In [28]:
svdcv = make_pipeline(TruncatedSVD(1000), Normalizer(copy=False))

In [29]:
begin = time.perf_counter()
cvsvd = svdcv.fit_transform(collocation_vector)
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 21.836217599999998


In [30]:
X_train = cvsvd

## Word Embedding Only

In [78]:
X_train = embedding

## IMS Collocation Vectors + Surrounding Words

In [57]:
transform_to_imscv_sw = lambda imscv, sw: csr_matrix(
    np.array(
        list(map(lambda i: [*imscv[i].toarray()[0], *sw[i].toarray()[0]], [i for i in range(imscv.shape[0])])),
        dtype=np.bool
    )
)

In [58]:
begin = time.perf_counter()
X_train = transform_to_imscv_sw(
    collocation_vectors,
    surrounding_words
)
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 250.5095411000002


In [78]:
X_train

<8721x238788 sparse matrix of type '<class 'numpy.bool_'>'
	with 301381 stored elements in Compressed Sparse Row format>

## TF-IDF Only

In [133]:
X_train = u_tfidf

## Unigram-Bigram TF-IDF Only

In [147]:
X_train = ub_tfidf

## LSA Only

In [139]:
X_train = lsa

## POS Tags Only

In [109]:
X_train = pos_tags

## POS Tags SVD

In [95]:
svdpos = make_pipeline(TruncatedSVD(80), Normalizer(copy=False))

In [96]:
possvd = svdpos.fit_transform(pos_tags)

In [97]:
X_train = possvd

## Surrounding Words SVD + POS Tags SVD

In [116]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*possvd[i], *swsvd[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 3.57551219999732


## IMS Collocation Vectors + POS Tags SVD

In [118]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*possvd[i], *imscvsvd[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 17.504873400001088


## It Makes Sense's Features Set SVD
The It Makes Sense's disambiguator system uses collocation vectors, surrounding words, and POS tags

In [120]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*possvd[i], *imscvsvd[i], *swsvd[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 18.15229390001332


## Wikipedia Word Embedding + POS Tags SVD

In [98]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*possvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 0.5411793000002945


## Wikipedia Word Embedding + Surrounding Words SVD

In [107]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*swsvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 3.6643506000000343


## Wikipedia Word Embedding + IMS Collocation Vectors SVD

In [123]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*imscvsvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 14.905517000000145


## Wikipedia Word Embedding + IMS Collocation Vectors SVD + Surrounding Words SVD

In [125]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*imscvsvd[i], *swsvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 37.737581800000044


## Wikipedia Word Embedding + IMS Collocation Vectors SVD + POS Tags SVD

In [127]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*imscvsvd[i], *possvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 35.42523809999875


## Wikipedia Word Embedding + Surrounding Words SVD + POS Tags SVD

In [129]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*swsvd[i], *possvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 1.7706331999997929


## Iacobacci, et. al Features Set SVD
But the word embedding is not transformed by SVD algorithm

In [131]:
begin = time.perf_counter()
X_train = np.array(list(map(lambda i: [*imscvsvd[i], *swsvd[i], *possvd[i], *embedding[i]], [i for i in range(len(dataset))])))
print('elapsed time:', time.perf_counter() - begin)

elapsed time: 17.88977089999935


## Labels

In [32]:
annotated_words = set(dataset.kata)

In [33]:
mappers = dict()
for w in annotated_words:
    possible_sense = set(dataset.query('kata == "{}"'.format(w)).sense)
    mappers[w] = []
    for sense, i in zip(list(possible_sense),  [n for n in range(len(possible_sense))]):
        mappers[w].append((sense, i))

In [34]:
y_train = np.array([list(filter(lambda m: m[0] == sense, mappers[kata]))[0][1] for sense, kata in zip(dataset.sense, dataset.kata)])

# Training

Dummy classifier: always choose the most frequent sense

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [36]:
classifier = {w: None for w in annotated_words}

In [37]:
classification_report([0,1,1], [0,1,0], output_dict=True)['macro avg']['f1-score']

0.6666666666666666

In [38]:
'''
Select best parameter using k-fold cross validation
'''
def train(X, y, clf, possible_param, fold=3):
    clf = GridSearchCV(clf, possible_param, cv=fold, n_jobs=7, iid=False)
    clf.fit(X, y)
    label_counts = np.bincount(y)
    most_freq_label = np.argmax(label_counts)
    print()
    print('Cross validation accuracy:', clf.best_score_)
    dummy_score = label_counts[most_freq_label] / len(y)
    print('Dummy classifier accuracy: ', dummy_score)
    print_param(clf.best_params_)
    return (clf.best_estimator_, clf.best_score_, dummy_score)

def train_f1(X, y, clf, possible_param, fold=3):
    clf = GridSearchCV(clf, possible_param, cv=fold, n_jobs=7, iid=False, scoring='f1_macro')
    clf.fit(X, y)
    label_counts = np.bincount(y)
    most_freq_label = np.argmax(label_counts)
    print()
    print('Training f1-score:', classification_report(y, clf.predict(X), output_dict=True)['macro avg']['f1-score'])
    print('Cross validation f1-score:', clf.best_score_)
    dummy_score = classification_report(y, [most_freq_label for i in y], output_dict=True)['macro avg']['f1-score']
    print('Dummy classifier f1-score: ', dummy_score)
    print_param(clf.best_params_)
    return (clf.best_estimator_, clf.best_score_, dummy_score)

In [39]:
def print_param(param):
    print('Best parameters:')
    for p in param:
        print(p, ':', param[p])

In [40]:
def train_all(clf, possible_param, fold=5, algorithm_name=''):
    print(algorithm_name)
    scores = []
    dummy_scores = []
    for w in classifier.keys():
        print('==================================')
        print(w)
        indexes = list(dataset.query('kata == "{}"'.format(w)).index)
        best_clf, best_score, dummy_score = train(X_train[indexes], y_train[indexes], clf, possible_param, fold)
        scores.append(best_score)
        dummy_scores.append(dummy_score)
        classifier[w] = best_clf
        print('----------------------------------')
    print('Cross validation macro average accuracy:', sum(scores)/len(scores))
    print('Dummy classifier macro average accuracy:', sum(dummy_scores)/len(dummy_scores))

def train_all_f1(clf, possible_param, fold=5, algorithm_name=''):
    print(algorithm_name)
    scores = []
    dummy_scores = []
    for w in classifier.keys():
        print('==================================')
        print(w)
        indexes = list(dataset.query('kata == "{}"'.format(w)).index)
        best_clf, best_score, dummy_score = train_f1(X_train[indexes], y_train[indexes], clf, possible_param, fold)
        scores.append(best_score)
        dummy_scores.append(dummy_score)
        classifier[w] = best_clf
        print('----------------------------------')
    print('Cross validation macro average f1-score:', sum(scores)/len(scores))
    print('Dummy classifier macro average f1-score:', sum(dummy_scores)/len(dummy_scores))

In [41]:
y_train[list(dataset.query('kata == "{}"'.format('besar')).index)]

array([1, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 1, 3, 1, 2, 1, 2, 3, 2, 1, 3, 1,
       1, 3, 1, 2, 1, 1, 2, 2, 3, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 2, 1, 2, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 1, 3, 0, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1,
       0, 1, 3, 2, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 1, 1,
       3, 2, 2, 2, 1, 1, 1, 1, 2, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1,
       1, 1, 2, 0, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1,
       2, 1, 2, 2, 2])

## Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression

In [ ]:

train_all(
    LogisticRegression(),
    {'solver':['newton-cg'], 'max_iter':[10, 20, 50], 'multi_class': ['ovr', 'multinomial']},
    algorithm_name='Logistic Regression'
)

## Linear SVM

In [42]:
from sklearn.svm import LinearSVC
import time

In [132]:
begin = time.perf_counter()
train_all_f1(
    LinearSVC(),
    {'max_iter': [10, 20, 40], 'C':[0.25, 0.5, 1.0, 2.0, 4.0, 8.0]},
    algorithm_name='Linear SVM'
)
print('elapsed time:', time.perf_counter() - begin)

Linear SVM
dunia


C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Training f1-score: 0.9104030501089325
Cross validation f1-score: 0.38233029422684595
Dummy classifier f1-score:  0.1794871794871795
Best parameters:
C : 0.5
max_iter : 10
----------------------------------
tinggi


C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Training f1-score: 0.9730847574139373
Cross validation f1-score: 0.5534184528921371
Dummy classifier f1-score:  0.10232558139534884
Best parameters:
C : 4.0
max_iter : 10
----------------------------------
mengandung


...

C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Training f1-score: 1.0
Cross validation f1-score: 0.4885371744775493
Dummy classifier f1-score:  0.3182674199623352
Best parameters:
C : 0.25
max_iter : 40
----------------------------------
membawa


C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Training f1-score: 0.9373365959681467
Cross validation f1-score: 0.31478238847986745
Dummy classifier f1-score:  0.05442176870748299
Best parameters:
C : 0.25
max_iter : 20
----------------------------------
harapan

Training f1-score: 0.958223626470072
Cross validation f1-score: 0.5829935145420114
Dummy classifier f1-score:  0.2818428184281843
Best parameters:
C : 1.0
max_iter : 20
----------------------------------
Cross validation macro average f1-score: 0.6345916016333781
Dummy classifier macro average f1-score: 0.2526642298604586
elapsed time: 256.5487159000004


C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\yerla\.conda\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [59]:
classification_report(
    y_train[list(dataset.query('kata == "{}"'.format('kunci')).index)], 
    classifier['kunci'].predict(X_train[list(dataset.query('kata == "{}"'.format('kunci')).index)]),
    output_dict=True
)

{'0': {'precision': 0.84,
  'recall': 0.9130434782608695,
  'f1-score': 0.8749999999999999,
  'support': 46},
 '1': {'precision': 0.9433962264150944,
  'recall': 0.8064516129032258,
  'f1-score': 0.8695652173913043,
  'support': 62},
 '2': {'precision': 0.8181818181818182,
  'recall': 1.0,
  'f1-score': 0.9,
  'support': 27},
 '3': {'precision': 1.0,
  'recall': 0.9166666666666666,
  'f1-score': 0.9565217391304348,
  'support': 12},
 'accuracy': 0.8843537414965986,
 'macro avg': {'precision': 0.9003945111492282,
  'recall': 0.9090404394576904,
  'f1-score': 0.9002717391304347,
  'support': 147},
 'weighted avg': {'precision': 0.892663096113231,
  'recall': 0.8843537414965986,
  'f1-score': 0.8839544513457556,
  'support': 147}}